In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics

# Establish Database Connection

In [2]:
import psycopg2

conn =psycopg2.connect("dbname=postgres user=postgres password=password host=35.236.196.171")

In [3]:
df = pd.DataFrame()
for chunk in pd.read_sql("SELECT * from events ORDER by random() LIMIT 1300000", con=conn, chunksize=1000):
    df = df.append(chunk)

# Data Cleaning

In [11]:
def clean_zip(x):
    if x == float('nan') or len(str(x)) == 5:
        return x
    elif len(str(x)) == 10:
        return x[:-5]
    else:
        return False
      
zip_seq = df["zip"].apply(clean_zip) #non-U.S. zip as False
df = df[zip_seq != False]

# Features

## List of Features

In [19]:
#dates
dates = df['event_date']
#print(dates)

In [13]:
#rsvp %
yes_percent = df["rsvp_yes"]/df["guests"]
no_percent = df["rsvp_no"]/df["guests"]
maybe_percent = df["rsvp_maybe"]/df["guests"]
undecided_percent = df["rsvp_undecided"]/df["guests"]

In [14]:
# time of the day
times = [date.to_pydatetime(date).hour for date in dates]
day_part = ['Morning' if (time >= 0 and time <= 11)
            else 'Afternoon' if (time >= 12 and time <= 17)
            else 'Evening' if (time >= 8 and time <= 23)
            else 'Unknown'
            for time in times]
#print(day_part)

In [15]:
#seasons
months = [date.to_pydatetime(date).month for date in dates]
seasons = [(month%12 + 3)//3 for month in months]
seasons_text = ['Winter' if s == 1 
                else 'Spring' if s == 2 
                else 'Summer' if s == 3 
                else 'Fall' if s == 4 
                else 'Unknown' for s in seasons]
#print(seasons_text)

In [16]:
#create event diff
created_dates = df['created_date']
created_dates = [create_date.to_pydatetime(create_date) for create_date in created_dates]
event_dates = df['event_date']
event_dates = [event_date.to_pydatetime(event_date) for event_date in event_dates]
create_event_diff = [event - created for event, created in zip(event_dates, created_dates)]
create_event_diff = [td.days for td in create_event_diff]
#print(create_event_diff)

In [17]:
#mail event diff
mail_dates = df['sent_on']
mail_dates = [mail_date.to_pydatetime(mail_date) for mail_date in mail_dates]
mail_event_diff = [event - mail for event, mail in zip(event_dates, mail_dates)]
mail_event_diff = [td.days for td in mail_event_diff]
#print(mail_event_diff)

In [18]:
#create mail diff
create_mail_diff = [created - mail for created, mail in zip(created_dates, mail_dates)]
create_mail_diff = [td.days for td in create_mail_diff]

In [20]:
#zip-income table
from uszipcode import SearchEngine,Zipcode
search = SearchEngine(simple_zipcode=True)
result = search.by_median_household_income(returns=0)
#to specify range,use lower/upper = ""
#result = search.by_median_household_income(lower=100000,upper=150000,returns=0)
#output to lists
zip_list = []
income_list = []

for zipcode in result:
    zip_list.append(zipcode.to_dict()['zipcode'])
    income_list.append(zipcode.to_dict()['median_household_income'])
    #rent_list.append(zipcode.to_dict()['monthly_rent_including_utilities_3plus_b'])
#convert to df
zip_df = pd.DataFrame({
    'zipcode':zip_list,
    'median_household_income':income_list
})

In [22]:
df.head()

,event_id,host_id,event_type,event_category,correspondence,event_category_group,seasonal,event_date,sent_on,created_date,location,zip,premium_vs_free,template_name,guests,rsvp_yes,rsvp_no,rsvp_maybe,rsvp_undecided
0,1213613,1012281,birthday_forkids,Birthday for Kids,False,Birthday,False,2018-11-04 13:00:00,2018-10-11 16:07:15.000000,2018-10-01 20:00:26.178694,Home,80138,Free,newchalkballoons_uyo,19,11,3,1,4
1,8891192,4552157,save_the_date,Save the Date,False,Wedding or related,False,2017-02-19 14:00:00,2017-01-30 18:34:02.000000,2017-01-29 03:41:08.706280,Church/Religious Institution,32561,Free,eaglescouts,50,9,3,0,38
2,6811809,1302872,holiday_party,Holiday Party,False,Winter Holidays,True,2015-12-12 18:00:00,2015-11-28 19:02:21.684760,2015-11-27 15:20:11.788110,Unidentified,60540,Free,ohwhatberryfun,59,24,19,3,13
3,371264,344035,halloween,Halloween,False,Halloween,True,2018-10-27 18:00:00,2018-10-06 18:23:01.000000,2018-10-05 01:22:06.125188,Home,60439,Free,halloweenspiders_ani,51,16,17,3,15
4,3671638,2487986,birthday_forkids,Birthday for Kids,False,Birthday,False,2015-12-05 14:05:00,2015-11-07 23:11:27.000000,2015-11-07 22:52:29.489110,Recreation Centers,80016,Free,oceansoffun,18,8,3,1,6


## Feature Selection

### Correlation

['Fall',
 'Winter',
 'Winter',
 'Fall',
 'Winter',
 'Fall',
 'Winter',
 'Fall',
 'Spring',
 'Fall',
 'Fall',
 'Winter',
 'Spring',
 'Winter',
 'Fall',
 'Fall',
 'Winter',
 'Fall',
 'Fall',
 'Winter',
 'Winter',
 'Fall',
 'Summer',
 'Winter',
 'Winter',
 'Winter',
 'Fall',
 'Winter',
 'Spring',
 'Winter',
 'Winter',
 'Spring',
 'Summer',
 'Fall',
 'Summer',
 'Winter',
 'Summer',
 'Summer',
 'Fall',
 'Spring',
 'Spring',
 'Summer',
 'Fall',
 'Summer',
 'Winter',
 'Fall',
 'Fall',
 'Spring',
 'Summer',
 'Fall',
 'Fall',
 'Spring',
 'Fall',
 'Summer',
 'Winter',
 'Summer',
 'Winter',
 'Spring',
 'Winter',
 'Spring',
 'Fall',
 'Summer',
 'Summer',
 'Winter',
 'Spring',
 'Summer',
 'Winter',
 'Fall',
 'Winter',
 'Summer',
 'Spring',
 'Fall',
 'Summer',
 'Spring',
 'Fall',
 'Fall',
 'Summer',
 'Summer',
 'Fall',
 'Fall',
 'Fall',
 'Winter',
 'Fall',
 'Summer',
 'Winter',
 'Winter',
 'Fall',
 'Spring',
 'Winter',
 'Fall',
 'Fall',
 'Spring',
 'Spring',
 'Fall',
 'Winter',
 'Spring',
 'Winter',